#### Setup and load torch

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
import sys,cv2
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from Utils.utils import *
from Utils.f1_score import *
from ipywidgets import interact
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

import torch
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

import timeit

print('Done.')

Done.


In [2]:
%%time

#Load training data
import deepdish as dd

data_folder = 'D:/data/HPA/all/'

d = dd.io.load(data_folder+'poi_0.h5')
X = d['X'].astype(np.float32) / 255.0 # torch likes float images

y = d['labels']
print("Done.")

Done.
Wall time: 10.6 s


In [11]:
from Models.simpleConvNet import *
#Setup resnet model and optimizer

# net = models.resnet18(pretrained=False)
# net.fc = nn.Linear(51200, 28) #adapt last layer to allow larger input images
# net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

net = simpleConvNet()

#BCE Loss
weights = np.asarray(y.shape[0]/np.sum(y,axis=0))
weights[15] = y.shape[0]
print("Weights = ",weights) # we weight classes given their skewed distribution
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(weights,dtype=torch.float))

optimizer = optim.Adam(net.parameters(),lr=0.00005)

print("Using GPU:",torch.cuda.is_available())
print("Using device ",torch.cuda.get_device_name(0))

print("Done.")

Weights =  [3.02571861e-04 2.39234450e-03 9.14913083e-04 2.07900208e-03
 1.80505415e-03 1.31926121e-03 2.90697674e-03 1.19474313e-03
 8.33333333e-02 5.55555556e-02 8.33333333e-02 3.03030303e-03
 4.52488688e-03 6.41025641e-03 2.91545190e-03 1.00000000e+00
 5.64971751e-03 1.33333333e-02 3.47222222e-03 2.13675214e-03
 1.75438596e-02 8.65051903e-04 3.80228137e-03 1.12485939e-03
 1.05263158e-02 4.29922614e-04 1.06382979e-02 1.66666667e-01]
Using GPU: True
Using device  GeForce GTX 1070
Done.


In [12]:
# Start training

epochs = 1
batch_size = 100
time_per_epoch = 0

iterations_per_epoch = round(30000 / batch_size)
runtime = 0
partNr = 1

for epoch in range(epochs):
    running_loss = 0
    running_f1 = 0
    
    current_buffer_size = X.shape[0]
    actual_idx = 0
    for i in range(iterations_per_epoch):
        if actual_idx*batch_size > y.shape[0]:
            print("Loading data part " + str(partNr))
            d = dd.io.load(data_folder+'poi_'+str(partNr)+'.h5')
            X = d['X'].astype(np.float32) / 255.0 # torch likes float images
            y = d['labels']
            actual_idx = 0
            partNr += 1
            print("Done.")
        
        start = timeit.default_timer() #measure time
        
        X_batch = torch.tensor(X[actual_idx:actual_idx+batch_size].transpose(0,3,1,2))
        y_batch = torch.tensor(y[actual_idx:actual_idx+batch_size].astype(np.float32),dtype=torch.float)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(X_batch)
        
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        #compute F1 scores
        act = torch.sigmoid(outputs)
        
        label = y_batch.detach().numpy().astype(np.bool)
        logits = act.detach().numpy() > 0.5
        print("Targets in batch = ",np.sum(label),"Predicted targets = ",np.sum(logits))
        running_f1 += f1_score(label,logits)
        
        #measure runtime
        stop = timeit.default_timer()
        time_per_epoch = 0.5 * time_per_epoch + 0.5 * (stop-start) * iterations_per_epoch
        runtime += (stop-start)
        #print performance metrics
        N = ((i+1)*batch_size)
        print('[epoch = (%d/%d), iteration = (%3d/%d), time = %3ds, est. time per epoch = %5ds] \t loss = %.5f ## F1 = %.5f'\
              %(epoch + 1, epochs,i + 1, iterations_per_epoch, runtime, time_per_epoch, running_loss / N, running_f1 / N))
        
        actual_idx += 1
        
    
    

..\Models\simpleConvNet.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.fc1(x)


RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #2 'target'